In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import os

# from embrapa_api.preprocessing.preprocessors import BasePreprocessor
from embrapa_api.config import CSV_FILES_FOLDER

In [3]:
try:
    2 + "2"
    data = (
        pd.read_csv(
            'http://vitibrasil.cnpuv.embrapa.br/download/Comercio.csv',
            sep=';'
        )
    )
except Exception as e:
    data = pd.read_csv(
        os.path.join(CSV_FILES_FOLDER, 'comercializacao_vinho/Comercio.csv'),
        sep=';'
    )
    print(e) # trocar por logging

unsupported operand type(s) for +: 'int' and 'str'


In [14]:
idx = [id for id in data[data["Produto"].str.isupper()].index]
idx = sorted(set(idx + [id + 1 for id in idx]))

In [15]:
idx

[0, 1, 4, 5, 8, 9, 10, 11, 14, 15, 18, 19, 24, 25, 26]

In [16]:
data.iloc[idx]

,id,control,Produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310
5,6,vm_Tinto,Tinto,435354,428927,624499,783508,1616144,2050960,4450570,...,15354938,15572632,15228514,12021684,11150517,11433702,18202453,19337862,15258778,12450606
8,9,VINHO FRIZANTE,VINHO FRIZANTE,466480,358116,356340,327865,270641,301293,248456,...,1893469,1836167,1727386,1586985,1638337,1826433,2557585,3696762,2875864,2843600
9,10,VINHO ORGÂNICO,VINHO ORGÂNICO,0,0,0,0,0,0,0,...,0,0,0,0,0,2554,10718,18686,14947,9123
10,11,VINHO ESPECIAL,VINHO ESPECIAL,7325789,7824931,8737605,9693953,7638484,10260445,12392725,...,7231,593,331,0,0,0,0,0,0,0
11,12,ve_Tinto,Tinto,4730346,4802828,4347706,5017487,3763509,4815235,4819463,...,7231,593,331,0,0,0,0,0,0,0
14,15,ESPUMANTES,ESPUMANTES,2743678,2907289,3523335,3625230,3442734,3691348,4204666,...,17191075,18897144,16945710,17583283,18218375,22759859,22610762,31242697,29525942,29381635
15,16,es_Espumante_Moscatel,Espumante Moscatel,0,0,0,0,0,0,0,...,4588465,5010704,4507467,5561181,6526075,8997187,9298571,12240059,12204315,9771698


In [17]:
TIPO_PRODUTO_MAP = {
    "vm": "Vinho de Mesa",
    "ve": "Vinho Especial",
    "es": "Espumante",
    "su": "Suco de Uva",
    "ou": "Outros Vinhos"
}

In [19]:
rf_comercializacao = (
    data.melt(
        id_vars=["id", "Produto", "control"],
        var_name="ano",
        value_name="comercializacao_L",
    )
    .rename(
        columns={
            "id": "ID_PRODUTO",
            "Produto": "NM_PRODUTO",
            "control": "NM_CONTROLE",
            "ano": "DT_ANO",
            "comercializacao_L": "VR_COMERCIALIZACAO_L",
        }
    )
    .astype(
        {
            "ID_PRODUTO": int,
            "NM_PRODUTO": str,
            "NM_CONTROLE": str,
            "DT_ANO": str,
            "VR_COMERCIALIZACAO_L": float,
        }
    )
    .sort_values(by=["ID_PRODUTO", "DT_ANO"])
)
rf_comercializacao["NM_PRODUTO"] = (
    rf_comercializacao["NM_PRODUTO"].apply(unidecode).str.title()
)
rf_comercializacao["TIPO_PRODUTO"] = (
    rf_comercializacao["NM_CONTROLE"].str.split(
        "_").str[0].map(TIPO_PRODUTO_MAP)
)


rf_comercializacao = rf_comercializacao.query("TIPO_PRODUTO.notnull()").drop(
    columns=["NM_CONTROLE"]
)

In [20]:
rf_comercializacao.head()

,ID_PRODUTO,NM_PRODUTO,DT_ANO,VR_COMERCIALIZACAO_L,TIPO_PRODUTO
1,2,Tinto,1970,83300735.0,Vinho de Mesa
63,2,Tinto,1971,98522869.0,Vinho de Mesa
125,2,Tinto,1972,101167932.0,Vinho de Mesa
187,2,Tinto,1973,98196747.0,Vinho de Mesa
249,2,Tinto,1974,77167303.0,Vinho de Mesa
